### Multi-camera recording to CSV 128px

In [1]:
import cv2
import numpy as np
import pandas as pd
import os
import glob

In [2]:
recording_dir = 'recordings40px011/'
amount_of_pixels = '40px' # _AMOUNT OF PIXELS_
sequence_number = '011' # _SEQUENCE NUMBER_, a number that is unique to the recording sequence

In [3]:
mp4_files = glob.glob(recording_dir + '*.mp4')

if not mp4_files:
    raise ValueError('No mp4 files found in the recording directory')

df = pd.DataFrame()

lower_red = np.array([0, 0, 100], dtype=np.uint8)
upper_red = np.array([75, 75, 255], dtype=np.uint8)

i=0

for video in mp4_files:
        
    cap = cv2.VideoCapture(video)
    mean_x = []
    mean_y = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        # create a mask for the red pixels in the frame
        mask = cv2.inRange(frame, lower_red, upper_red)
        # apply the mask to the frame so that only the red pixels are shown
        detected_ball = cv2.bitwise_and(frame, frame, mask=mask)    
        # get all the coordinates of red pixels in the frame
        red_coordinates = np.column_stack(np.where((frame[:, :, 0] >= lower_red[0]) & (frame[:, :, 0] <= upper_red[0]) &
        (frame[:, :, 1] >= lower_red[1]) & (frame[:, :, 1] <= upper_red[1]) &
        (frame[:, :, 2] >= lower_red[2]) & (frame[:, :, 2] <= upper_red[2])
        ))    
        # flip the y coordinates to match the origin at the bottom left
        red_coordinates[:, 0] = frame.shape[0] - red_coordinates[:, 0]
        if red_coordinates.size == 0:
            # add NaN if no red ball is detected
            mean_x.append('NaN')
            mean_y.append('NaN')
        else:
            mean_x.append(np.mean(red_coordinates[:, 1]))
            mean_y.append(np.mean(red_coordinates[:, 0]))
    
    # add the median x and y coordinates to the dataframe
    df[f'u{i}'] = mean_x
    df[f'v{i}'] = mean_y
    cap.release()
    
    i+=1
# Make sure the 'data' directory exists
if not os.path.exists('data'):
    os.makedirs('data')

# Save the dataframe to a CSV file
output_file_name_of_red_ball_coordinates = (
        'red_ball_coordinates_' + amount_of_pixels + '_' +  sequence_number +'.csv')
df.to_csv(f'data/' + output_file_name_of_red_ball_coordinates , index=False)